In [1]:
%matplotlib inline

import os
import pandas as pd
import math
import re
import seaborn as sns
import numpy as np
import glob
from functools import reduce

import csv

from matplotlib import pyplot as plt
import matplotlib.dates as md
from datetime import datetime,timedelta

In [2]:
import matplotlib

plt.rcParams['figure.figsize'] = 50, 10
font = {'size'   : 30}
matplotlib.rc('font', **font)

In [3]:
import random

# Input

n = number of users

m = number of items

k = number of latent factors

g = number of agents

sigma = standard deviation of distribution

A = list of user vectors (a1...an)

B = list of item vectors (b1...bm)

In [20]:
n = 3
m = 10

k = 3
g = 2
sigma = 1.0

delta
l = length of recommendations list

alpha = how big of a portion of items are selected - constant to be multiplied by lenght 

U = users latent factor matrix - scale of -1 to 1

V = items latent factor matrix - binary (0/1)

In [18]:
l = 10
alpha = 0.5

# U = np.array(
#     [[0.0, 0.2, 0.3],
#     [-1.0, 0.0, 0.5],
#     [-0.6, 1.0, -3.0]])

# V = np.array(
#     [[0,1,0],
#     [1,0,0],
#     [1,0,1]])

# Users/items Generation

In [6]:
# theoretically one function with a switch

In [48]:
def generate_users(n: int, feature_propensities=None) -> list:
    if feature_propensities is None:
        feature_propensities = [(0.0, 1.0)] * k
    a = []
    for i in range(n):
        ai = []
        for i in range(k):
            mu_i = feature_propensities[i][0]
            sigma_i = feature_propensities[i][1]
            aij = round(np.random.normal(loc=mu_i, scale=sigma_i), 2)
            ai.append(aij)
        a.append(ai)
            
    return a

In [30]:
def generate_items(m: int, feature_propensities=None) -> list:
    if feature_propensities is None:
        feature_propensities = [0.5] * k
    b = []
    for i in range(m):
        bi = []
        for i in range(k):
            feature_p = feature_propensities[i]
            bij = random.choices([0, 1], weights=(1 - feature_p, feature_p))
            bi += bij
        b.append(bi)
            
    return b

In [49]:
A = generate_users(n=n)
B = generate_items(m=m)

In [50]:
A

[[1.87, 0.67, -0.39], [-1.18, 0.87, -0.09], [0.37, -1.05, 0.53]]

In [39]:
B

[[0, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [0, 1, 1],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 0, 0],
 [1, 1, 0],
 [1, 0, 1]]

In [51]:
A = generate_users(n=n, feature_propensities=[(0.0, 1.0), (0.5, 1.0), (0.9, 1.0)])

In [52]:
A

[[0.54, -0.78, 1.88], [-0.09, 0.47, 0.84], [-2.0, -0.41, 1.94]]

In [41]:
B = generate_items(m=m, feature_propensities=[0.1, 0.5, 0.9])

In [42]:
B

[[0, 0, 1],
 [0, 1, 1],
 [0, 1, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [0, 1, 1],
 [0, 0, 1]]

# Factors Generation

In [53]:
def create_user_factors(k: int, g: int, A: list, sigma: float) -> np.array:
        U = []
    
        for ai in A:
            ui = []
            for j,aij in enumerate(ai):
                if j+1 > g:    # not an agent factor
                    uij = np.random.normal(loc=0.0, scale=sigma, size=None)
                else:      # an agent factor
                    uij = np.random.normal(loc=aij, scale=sigma, size=None)
                ui.append(uij)
            U.append(ui)
        
        
#         TODO: does it only take g first columns? only the ones with associated agent?
        
        U = np.array(U)
        return U

In [54]:
U = create_user_factors(k,g,A,sigma) # users
V = create_user_factors(k,g,B,sigma) # items

In [55]:
U

array([[ 1.24533955,  0.29017701,  1.0741475 ],
       [-0.89462397, -0.42090447,  0.59256929],
       [-2.71469512,  0.51461819,  0.29126728]])

In [56]:
V

array([[-0.35442687,  0.47502763,  1.35806228],
       [ 2.65932954,  1.0879131 , -0.9979858 ],
       [-0.10759669,  1.6630272 ,  1.10070368],
       [ 0.8692034 ,  1.62714579,  0.00879153],
       [-0.33483999, -0.54786854,  0.71128935],
       [ 1.36498284, -1.42940606,  0.48044305],
       [ 0.43710815, -1.14858875,  0.67854883],
       [ 1.23533753, -0.37251224,  0.39145879],
       [-0.83172551,  1.04328909, -0.86121544],
       [ 2.40842204,  2.63912738, -0.33975952]])

# Ratings Generation

In [57]:
def generating_ratings(alpha: float, l:l, U: np.array, V: np.array):

    delta = int(l*alpha)
    Vl = V.shape[0]
    
    with open('ratings.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        fields = ["user_id", "item_id", "score"]
        writer.writerow(fields)
    
    for userId,user in enumerate(U):
        
        userVIds = np.random.choice(Vl, size=delta, replace=False)
        userV = V[userVIds, :]

        userScores = []
        for itemId,item in zip(userVIds,userV):
            score = np.dot(user, item)
            userScores.append(tuple((itemId, score)))
        userScores.sort(key = lambda x: x[1], reverse = True) 

        for i,userScore in enumerate(userScores):
            if i >= l: # only choose L first scores
                break
            with open('ratings.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile, delimiter=',')
                writer.writerow([userId, userScore[0], userScore[1]])

In [58]:
generating_ratings(alpha=alpha,l=l,U=U,V=V)